# GLasso

The objective function is $$max_\Theta lndet\Theta - tr(S\Theta)-\rho\|\Theta\|_1,$$

where $\Theta$ is the precision (inverse covariance) matrix, $S=\frac1nX^TX$ is empirical sample covariance, $\|\Theta\|_1$ is the sum of absolute values of elements in $\Theta$.

The subgradient equation for objective function is $$\Sigma-S-\rho\Gamma=0,$$

where $\Sigma$ is the covariance matrix, $\Gamma_{ij}=sign(\Theta_{ij})$ if $\Theta_{ij}\neq0$, $\Gamma_{ij}\in[-1,1]$ if $\Theta_{ij}=0$.

For diagonal elements, $\Theta_{ii}>0$ then $\Gamma_{ii}=1$, so $\Sigma_{ii}=S_{ii}+\rho$.

Consider 1 versus (p-1) partition over $p$ variables, let
$$
\left(\begin{array}{cc} \Sigma_{aa} & \Sigma_{ab} \\ \Sigma_{ba} & \Sigma_{bb} \end{array}\right)
\left(\begin{array}{cc} \Theta_{aa} & \Theta_{ab} \\ \Theta_{ba} & \Theta_{bb} \end{array}\right) = 
\left(\begin{array}{cc} I_{p-1} & \textbf{1} \\ \textbf{1}^T & 1 \end{array}\right)
$$
For the upper right block in the subgradient equation, $\Sigma_{ab}-S_{ab}-\rho\Gamma_{ab}=0$, whose solution is equivalent with another optimization problem $$min_{\beta} \frac12\|\Sigma_{aa}^{1/2} \beta - \Sigma_{aa}^{-1} S_{ab} \|^2+\rho\|\beta\|_1,$$
whose subgradient equation is $\Sigma_{aa}\beta - S_{ab}+\rho sign(\beta)=0$, where $\beta=\Sigma_{aa}^{-1}\Sigma_{ab}, sign(\beta)=-\Gamma_{ab}$.

###### Note the connection and difference between column-wise solving 1 versus (p-1) conditional regression by lasso and the glasso over whole precision matrix is that $p$ seperate conditional regression is lasso($S_{aa},S_{ab},\rho$), while glasso solves $p$ coupled lasso problems with the same $Sigma$ shared as lasso($\Sigma_{ab}, S_{ab}, \rho$).

  ###### Graphical Lasso algorithm
Let $W=\hat\Sigma$ be the estimated covariance matrix,  is as follows
1. $W=S+\rho I_p$, then the diagonal remains unchanged
2. Repeat $j=1,2,...,p,1,2,...,p,...$, for each column $j$ in $W$, solve $\hat\beta=$lasso($\Sigma_{ab}, S_{ab}, \rho$), update $W_{ab}=W_{aa}\hat\beta$, until convergence of $W$

Details of step 2 in algorithm:
1) circle through all the variables multiple times, for each $j$-th column in the original $W$ is switched to the last column then it is updated as the upper right block $W_{ab}$ is updated  
2) lasso($\Sigma_{ab}, S_{ab}, \rho$) is solved by coordinate descent, where each entry in $\beta$ is updated iteratively by $$\hat\beta_i \leftarrow ST\left( (S_{ab})_{i} - \Sigma_{k\neq i} (W_{aa})_{ki}\hat\beta_k, \rho \right) / { (W_{aa})_{ii} },$$ where ST is the soft-threshold operator $ST(x, \rho)=sign(x)(|x|-t)_+$  
3) Convergence of $W$ is verified by its average absolute change less than the multiplication between a fixed threshold $t=0.001$ and the average of absolute value of off-diagonal elements in empirical covariance matrix $S$  
4) After convergence of $W$, the estimated precision matrix can be obtained by $\hat\beta$ s, according to $\hat\Theta_{bb}=1/(W_{bb}-W_{ab}^T\hat\beta)$ and $\hat\Theta_{ab}=-\hat\beta\Theta_{bb}$.


In [1]:
# Set the ground truth of precision matrix (\Theta)
n_aux = 20
p = 10
set.seed(0)
X_aux = matrix( rnorm(n_aux*p), ncol=p ) # Auxillary data for generating truth of covariance
Sigma_true = cov(X_aux)
# Sigma_true
Theta_true = solve(Sigma_true)
Theta_true

1.27498518,-0.360013241,-0.13899649,-0.04053069,-0.108175498,-0.22897547,0.43052975,0.01709085,-0.39463630,0.02215637
-0.36001324,3.134190094,1.01737618,-0.77021533,-0.009104576,0.37902948,-0.44378911,0.56362750,0.11966498,-0.46572278
-0.13899649,1.017376182,1.38116548,-0.96414716,-0.030133155,0.05454111,-0.25264663,-0.01056241,0.05395003,-0.35386972
-0.04053069,-0.770215326,-0.96414716,3.52498851,0.656934400,-0.39441460,0.11595627,-0.58714774,0.03967201,-0.27566292
-0.10817550,-0.009104576,-0.03013316,0.65693440,1.878431591,0.14829337,-0.20895989,-0.30118765,0.03356446,-0.09224890
-0.22897547,0.379029484,0.05454111,-0.39441460,0.148293373,1.30772526,-0.04191247,0.11365076,-0.06444851,-0.14225305
0.43052975,-0.443789113,-0.25264663,0.11595627,-0.208959889,-0.04191247,1.32480422,-0.07923855,-0.22972662,-0.20156463
0.01709085,0.563627505,-0.01056241,-0.58714774,-0.301187650,0.11365076,-0.07923855,1.38270476,-0.05168962,0.31653789
-0.39463630,0.119664977,0.05395003,0.03967201,0.033564464,-0.06444851,-0.22972662,-0.05168962,0.93065700,-0.04880206
0.02215637,-0.465722778,-0.35386972,-0.27566292,-0.092248902,-0.14225305,-0.20156463,0.31653789,-0.04880206,1.77034700


In [2]:
# Generate data from true precision (true covariance)
n = 9
X = MASS::mvrnorm(n=n, mu=rep(0,p), Sigma=Sigma_true)
# Settings of glasso algorithm
rho = 0.1
t = 1e-4 # a fixed threshold
S = cov(X)*(n-1)/n # empirical

In [3]:
ave_S = mean(S[lower.tri(S)])
threshold = t * ave_S
max_iter = 100

# Step 1: diagonal of W remains unchanged
W = S + rho*diag(p)
# print(W)
# B = matrix(0, (p-1), p) # store all betas

# Step 2:
iter = 0
while(iter < max_iter){
  W_old = W
  for(i in 1:p){ # iterate column in covariance
#     W11 = S[-i, -i] # use approximation as column-wise lasso for test
    W11 = W[-i, -i]
    
    s12 = S[-i,i]

    w12 = W[-i, i]
    beta = solve(W11) %*% w12
#     beta = B[, i]
    
    # coordinate decent
    max_iter_lasso = 20
    iter_lasso = 0
    while(iter_lasso < max_iter_lasso){
      beta_old = beta
      for(j in 1:(p-1)){ # iterate entries in beta
        uj = s12[j]
        for(k in 1:(p-1)){
          if(k==j) next
          uj = uj - W11[k,j] * beta[k]

        }
#         cat(j, uj, '  ')
        if(abs(uj) > rho){
          beta[j] = (abs(uj) - rho) / W11[j,j]
          if(uj < 0) beta[j] = -beta[j]
        } 
        else{
          beta[j] = 0
        }
      }
#       cat('\n')
      iter_lasso = iter_lasso + 1
      
      change_lasso = max( abs(beta - beta_old) )
      if(change_lasso<t) break
    }
#     cat('iter_lasso: ', iter_lasso, '\n')
    
    w12 = W11 %*% beta
#     cat(i, 'beta: ', beta, '\n', '  w12:', w12,'\n')
    W[-i, i] = W[i, -i] = w12
#     B[, i] = beta
#     print(W)
  }
  
  iter = iter + 1
  change = max( abs(W-W_old) )
  if(change < threshold) break
  else{
    cat('iter', iter, change, '\n')
  }
}

iter 1 0.1000172 
iter 2 0.03564175 
iter 3 0.001287652 
iter 4 5.914316e-05 
iter 5 1.559448e-05 


In [4]:
Theta_glasso = solve(W)
round(Theta_glasso,8)

1.44131720,-0.05468838,0.05354554,-0.00000086,-0.56346842,0.17435579,0.32708832,-0.71906876,0.00000000,0.00152736
-0.05468838,2.24621882,0.52364729,-0.00000084,-0.00000017,-0.00000011,0.22789042,0.14648430,0.21936794,-0.38582536
0.05354554,0.52364729,2.04851276,-0.92237944,-0.00000013,-0.07054451,-0.07351795,-0.39155108,0.43813365,-0.24805204
-0.00000086,-0.00000084,-0.92237944,3.51590910,-0.32891514,-0.78028081,0.00000000,0.00000002,0.00000002,-0.42883836
-0.56346842,-0.00000017,-0.00000013,-0.32891514,3.10161854,-0.15999947,0.53073839,-0.27070848,-0.15020936,-0.00649441
0.17435579,-0.00000011,-0.07054451,-0.78028081,-0.15999947,1.10494806,-0.11286726,-0.09716585,-0.32038924,0.00000000
0.32708832,0.22789042,-0.07351795,0.00000000,0.53073839,-0.11286726,1.28585940,0.00000000,0.00000000,0.00000000
-0.71906876,0.14648430,-0.39155108,0.00000002,-0.27070848,-0.09716585,0.00000000,1.76994044,-0.03693921,0.28791870
0.00000000,0.21936794,0.43813365,0.00000002,-0.15020936,-0.32038924,0.00000000,-0.03693921,0.86196105,0.05180987
0.00152736,-0.38582536,-0.24805204,-0.42883836,-0.00649441,0.00000000,0.00000000,0.28791870,0.05180987,0.75370995


### Compare with the outcome of original implementaion of R package glasso (by Fortran internally)

In [5]:
glasso::glasso(S, rho=rho)$wi

1.441318313,-0.05468993,0.05354667,0.0000000,-0.563458507,0.17435728,0.32708792,-0.71907706,0.00000000,0.001524034
-0.054688765,2.24621888,0.52364641,0.0000000,0.000000000,0.00000000,0.22788994,0.14648668,0.21936792,-0.385825164
0.053546259,0.52364592,2.04851200,-0.9223762,0.000000000,-0.07054298,-0.07351858,-0.39155091,0.43813384,-0.248052595
0.000000000,0.00000000,-0.92237507,3.5159082,-0.328924874,-0.78028279,0.00000000,0.00000000,0.00000000,-0.428838587
-0.563469507,0.00000000,0.00000000,-0.3289192,3.101608921,-0.15999533,0.53073901,-0.27071061,-0.15021093,-0.006493087
0.174353664,0.00000000,-0.07054566,-0.7802807,-0.159991881,1.10494852,-0.11286548,-0.09717106,-0.32038930,0.000000000
0.327090480,0.22789096,-0.07351854,0.0000000,0.530737365,-0.11286751,1.28585876,0.00000000,0.00000000,0.000000000
-0.719067311,0.14648621,-0.39155321,0.0000000,-0.270710191,-0.09716916,0.00000000,1.76994907,-0.03693925,0.287920282
0.000000000,0.21936760,0.43813413,0.0000000,-0.150208957,-0.32038916,0.00000000,-0.03693588,0.86196136,0.051809602
0.001529211,-0.38582500,-0.24805293,-0.4288401,-0.006491063,0.00000000,0.00000000,0.28791951,0.05180980,0.753710017
